In [4]:
import os
import pandas as pd
import numpy as np
import scipy.optimize as sc 
import plotly.graph_objects as go

# Import Data

In [5]:
# Import data from files
os.chdir('/Users/talhajamal/Desktop/Code/BigData1Project')
data = pd.read_csv("data/Returns_Data.csv")
data['date'] = pd.to_datetime(data['date'], dayfirst=False)
characteristics = pd.read_csv("data/Stock_Characteristics_Data.csv")
dictionary = pd.read_excel("data/StockDataDictionary.xlsx")

# Create new dataframes
prices = data.pivot(index='date', columns='ticker', values='PRC')
volume = data.pivot(index='date', columns='ticker', values='VOL')
returns = data.pivot(index='date', columns='ticker', values='RET')
#returns = returns * 100 # Scale returns to percentage
# Summary of Returns
returns_summary = returns.describe()
shares_outstanding = data.pivot(index='date', columns='ticker', values='SHROUT')
value_weighted_returns = data.pivot(index='date', columns='ticker', values='vwretd')
equal_weighted_returns = data.pivot(index='date', columns='ticker', values='ewretd')
tickers = prices.columns # List of Tickers

# Functions

In [20]:
def portfolioPerformance(weights, meanReturns, covMatrix):
    """ Calculate Portfolio Performance"""
    annualizedReturns = np.sum(meanReturns*weights)*252
    annualizedStd = np.sqrt( np.dot(weights.T, np.dot(covMatrix, weights)) ) * np.sqrt(252)
    return annualizedReturns*100, annualizedStd*100

def portfolioVariance(weights, meanReturns, covMatrix):
    """ Return Portfolio Variance """
    return portfolioPerformance(weights, meanReturns, covMatrix)[1]

def portfolioReturn(weights, meanReturns, covMatrix):
    """ Return Portfolio Return """
    return portfolioPerformance(weights, meanReturns, covMatrix)[0]

def negativeSR(weights, meanReturns, covMatrix,riskFreeRate = 0):
    """ Calculate Negative Sharpe so that Optimizer works on it and minimizes it -> essentially maximising it"""
    annualizedRet, annualizedStd = portfolioPerformance(weights, meanReturns, covMatrix)
    return - ((annualizedRet - riskFreeRate)/annualizedStd)

def maxSR(meanReturns, covMatrix, riskFreeRate = 0, constraintSet = (0, 1)):
    """ Minimize the negative Sharpe Ratio -> Maximize it"""
    numAssets = len(meanReturns)
    args = (meanReturns, covMatrix, riskFreeRate)
    constraints = ({'type':'eq','fun': lambda x: np.sum(x) - 1 }) # Weights must sum up to 1
    bound = constraintSet
    bounds = tuple(bound for asset in range(numAssets)) # For every asset have this bound
    result = sc.minimize(negativeSR, x0=numAssets*[1./numAssets], args=args, 
                         method='SLSQP', bounds=bounds, constraints=constraints)
    return result

def minimizeVariance(meanReturns, covMatrix, riskFreeRate = 0, constraintSet = (0, 1)):
    """ Minimize the portfolio variance by altering the weights/allocations of assets in the portfolio"""
    numAssets = len(meanReturns)
    args = (meanReturns, covMatrix)
    constraints = ({'type':'eq','fun': lambda x: np.sum(x) - 1 }) # Weights must sum up to 1
    bound = constraintSet
    bounds = tuple(bound for asset in range(numAssets)) # For every asset have this bound
    result = sc.minimize(portfolioVariance, x0=numAssets*[1./numAssets], args=args, 
                         method='SLSQP', bounds=bounds, constraints=constraints)
    return result

def meanVarianceOptimiser(meanReturns, covMatrix, returnTarget, constraintSet=(0,1)):
    """ For each return target, optimize the portfolio for minimum variance"""
    numAssets = len(meanReturns)
    args = (meanReturns, covMatrix)
    constraints = ({'type':'eq', 'fun':lambda x: portfolioReturn(x, meanReturns, covMatrix) - returnTarget},
                   {'type':'eq','fun': lambda x: np.sum(x) - 1 }
                   )
    bound = constraintSet
    bounds = tuple(bound for asset in range(numAssets))
    optimizedResults = sc.minimize(portfolioVariance, numAssets*[1./numAssets], args=args, 
                                   method='SLSQP', constraints=constraints, bounds=bounds)
    return optimizedResults

def eqWeightPortfolioPerformance(meanReturns, covMatrix, returns):
    """ Return the Performance of an Equally Weighted Portfolio"""
    weights = np.array([1/100 for _ in returns])
    equalWeightedPortfolioReturns, equalWeightedPortfolioVolatility = portfolioPerformance(weights, meanReturns, covMatrix)
    equalWeightedPortfolioSR = (-1) * negativeSR(weights, meanReturns, covMatrix)
    print(f"Equal Weighted Portfolio Returns: {round(equalWeightedPortfolioReturns, 2)} %")
    print(f"Equal Weighted Portfolio Volatility: {round(equalWeightedPortfolioVolatility, 2)} %")
    print(f"Equal Weighted Portfolio SR: {round(equalWeightedPortfolioSR, 2)}")
    return equalWeightedPortfolioReturns, equalWeightedPortfolioVolatility, equalWeightedPortfolioSR
    
def maxSRPortfolioPerformance(meanReturns, covMatrix):
    """ Return the Performance of the Maximum SR Portfolio"""
    maxSRPortfolioPerformance = maxSR(meanReturns, covMatrix)
    maxSRPortfolioWeights = maxSRPortfolioPerformance.x
    maxSRPortfolioReturns, maxSRPortfolioVolatility = portfolioPerformance(maxSRPortfolioWeights, meanReturns, covMatrix)
    maxSRPortfolioSR = maxSRPortfolioPerformance.fun * (-1)
    print(f"Max SR Portfolio Returns: {round(maxSRPortfolioReturns, 2)} %")
    print(f"Max SR Portfolio Volatility: {round(maxSRPortfolioVolatility, 2)} %")
    print(f"Max SR Portfolio SR: {round(maxSRPortfolioSR, 2)}")
    return maxSRPortfolioReturns, maxSRPortfolioVolatility, maxSRPortfolioSR, maxSRPortfolioWeights

def minVariancePortfolioPerformance(meanReturns, covMatrix):
    """ Return the Performance of the Minimum Variance Portfolio"""
    minVariancePortfolioPerformance = minimizeVariance(meanReturns, covMatrix)
    minVariancePortfolioWeights = minVariancePortfolioPerformance.x
    minVariancePortfolioReturns, minVariancePortfolioVolatility = portfolioPerformance(minVariancePortfolioWeights, meanReturns, covMatrix)
    minVariancePortfolioSR = (-1) * negativeSR(minVariancePortfolioWeights, meanReturns, covMatrix)
    print(f"Min Variance Portfolio Returns: {round(minVariancePortfolioReturns, 2)} %")
    print(f"Min Variance Portfolio Volatility: {round(minVariancePortfolioVolatility, 2)} %")
    print(f"Min Variance Portfolio SR: {round(minVariancePortfolioSR, 2)}")
    return minVariancePortfolioReturns, minVariancePortfolioVolatility, minVariancePortfolioSR, minVariancePortfolioWeights

def EF_generator(minVariancePortfolioReturns, maxSRPortfolioReturns, meanReturns, covMatrix):
    """ Generate the List for the Efficient Frontier"""
    targetReturns = np.linspace(minVariancePortfolioReturns, maxSRPortfolioReturns, 30)
    efficientFrontier = []
    for target in targetReturns:
        efficientFrontier.append(meanVarianceOptimiser(meanReturns, covMatrix, returnTarget=target)['fun'])
    return efficientFrontier, targetReturns

def EF_graph(meanReturns, covMatrix, riskFreeRate = 0, constraintSet = (0, 1)):
    """ Return a Graph plotting the min vol, max sr, and efficient frontier"""
    maxSRPortfolioRet, maxSRPortfolioVol, maxSRPortfolioSR, maxSRPortfolioWeights = maxSRPortfolioPerformance(meanReturns, covMatrix)
    minVarPortfolioRet, minVarPortfolioVol, minVarPortfolioSR, minVariancePortfolioWeights = minVariancePortfolioPerformance(meanReturns, covMatrix)

    efficientList, targetReturns = EF_generator(minVarPortfolioRet, maxSRPortfolioRet, meanReturns, covMatrix)

    # Plot the Max SR
    MaxSharpeRatio = go.Scatter(
        name='Max Sharpe Ratio',
        mode='markers',
        x = [maxSRPortfolioVol],
        y = [maxSRPortfolioRet],
        marker = dict(color='red', size=14, line=dict(width=3, color='black') )
    )

    # Plot the Min vol
    MinVol = go.Scatter(
        name='Min Volatility',
        mode='markers',
        x = [minVarPortfolioVol],
        y = [minVarPortfolioRet],
        marker = dict(color='green', size=14, line=dict(width=3, color='black') )
    )

    # Plot the Efficient Frontier
    EF_curve = go.Scatter(
        name='Efficient Frontier',
        mode='lines',
        x = [round(ef_std, 2) for ef_std in efficientList],
        y = [round(target, 2) for target in targetReturns],
        line = dict(color='black', width=4, dash='dashdot')
    )

    data = [MaxSharpeRatio, MinVol, EF_curve]
    
    layout = go.Layout(
        title = 'Portfolio Optimization via a Mean Variance Optimiser',
        yaxis = dict(title='Annualized Returns (%)'),
        xaxis= dict(title='Annualized Volatility (%)'),
        showlegend= True,
        legend= dict(
                    x=0.7, 
                    y=0, 
                    traceorder='normal', 
                    bgcolor='#E2E2E2', 
                    bordercolor='black', 
                    borderwidth=2),
        width=800,
        height=600)
    
    fig = go.Figure(data=data, layout=layout)
    return fig.show()

# Mean Returns and CovMatrix

In [7]:
meanReturns = returns.mean()
covMatrix = returns.cov()

# Equal Weighted Portfolio

In [8]:
equalWeightedPortfolioRet, equalWeightedPortfolioVol, equalWeightedPortfolioSR = eqWeightPortfolioPerformance(meanReturns, 
                                                                                                              covMatrix, 
                                                                                                              returns)

Equal Weighted Portfolio Returns: 15.93 %
Equal Weighted Portfolio Volatility: 18.62 %
Equal Weighted Portfolio SR: 0.86


# Max Sharpe Ratio Portfolio

In [9]:
maxSRPortfolioRet, maxSRPortfolioVol, maxSRPortfolioSR, maxSRPortfolioWeights = maxSRPortfolioPerformance(meanReturns, covMatrix)

Max SR Portfolio Returns: 22.68 %
Max SR Portfolio Volatility: 15.51 %
Max SR Portfolio SR: 1.46


# Minimum Variance Portfolio

In [10]:
minVarPortfolioRet, minVarPortfolioVol, minVarPortfolioSR, minVariancePortfolioWeights = minVariancePortfolioPerformance(meanReturns, covMatrix)

Min Variance Portfolio Returns: 13.33 %
Min Variance Portfolio Volatility: 12.82 %
Min Variance Portfolio SR: 1.04


# Mean Variance Portfolio

In [28]:
EF_generator(minVarPortfolioRet, maxSRPortfolioRet, meanReturns, covMatrix)

[12.818748132709013,
 12.82246822484856,
 12.832419277206048,
 12.848318661294227,
 12.870165709244045,
 12.898034713667773,
 12.931900930103385,
 12.971716741186572,
 13.017407310900692,
 13.068834797868101,
 13.125900770473367,
 13.188588048824206,
 13.258200385097119,
 13.33450616018247,
 13.417167002607796,
 13.507110962388374,
 13.603999790370125,
 13.707349804897206,
 13.81685921938083,
 13.932379397355094,
 14.053862875690633,
 14.181706790215234,
 14.31604959171549,
 14.457347422253317,
 14.606327599117483,
 14.763089744897155,
 14.92759763802915,
 15.103770411722216,
 15.297944993299229,
 15.509766692328721]

In [21]:
EF_graph(meanReturns, covMatrix)

Max SR Portfolio Returns: 22.68 %
Max SR Portfolio Volatility: 15.51 %
Max SR Portfolio SR: 1.46
Min Variance Portfolio Returns: 13.33 %
Min Variance Portfolio Volatility: 12.82 %
Min Variance Portfolio SR: 1.04
